In [ ]:
"""

- Imoveis na periferia de São Paulo são subvalorizados?

- A ideia que norteia o projeto é entender se os imoveis que em raio de 30km do marco 0 da cidade de São Paulo são subvalorizados em comparação ao centro ou bairros nobres
- Para possibilitar uma analise de recomendação de investimento nesses imoveis com base em expectativa de lucro, tempo de retorno e analise de risco do mercado.
- Possiveis interessados na analise exploratoria: Startups do mercado imobiliario, Fundos Imobiliarios, Construtoras, Instituições financeiras que fornecem credito imobiliario

- analise baseada nos imoveis disponiveis nos sites Loft, Quinto Andar e Zap Imoveis

- a primeira missão seria conseguir as informações dos imoveis listados com [id_anuncio, endereco, preco, m2, quartos, banheiro, vaga de garagem, tipo imovel, novo/usado, rating atribuido por mim]
- se possivel obter alguma informação sobre a venda desses imoveis (talvez diario oficial para buscar tranferencias de imoveis)
- com base no endereço começamos a classificar se o imovel está em area periferica, neutra ou nobre
- pós classificação começamos a fazer as comparações de rating e preco de imoveis similares em locais diferentes
- tirar mais ideias sobre a situação imobiliaria de sao paulo
- tentar criar algoritmo que busca oportunidades de lucro 

-- linkar com problemas habitacionais da cidade
-- se possivel realizar uma regressao para comparar o quanto cada variavel impacta no preço do m2/total do imovel e nao se esquecer de adicionar uma variavel para segurança do bairro e outros fatores externos


WebScreaping 
Beautiful soup
Seleniun
Paralelização 

--quando fizer o webscraping procurar as <tags> que contenha apenas a info que eu quero
--para buscar diversas tags diferentes é interessante criar uma função com condicionais para cada type de informação dentro da tag

EXEMPLO
tags = ['p','h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7']
text_list = []
for element in soup.find_all(tags):
    text_list.append(element.text)
print('\n'.join(text_list))


"""

In [ ]:
"""TASK LIST 
- fazer o scraping de uma das variaveis do imovel em uma pagina simples [feito]
- criar uma lista dos links dos imoveis da tela principal



"""

# Imports e acesso ao banco de dados

In [1]:
# Imports que vou utilizar no codigo

import tqdm
import pandas as pd
import numpy as np

import requests
from datetime import datetime

from bs4 import BeautifulSoup
from lxml import etree

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys



In [ ]:
# Conexão com SQL para no futuro exportar meu Dataframe para SQL

import sqlalchemy
from sqlalchemy import create_engine 

db_server='pymysql'
user='root'
db_port = '3306'
password = 'password'
ip = 'localhost'
db_name = 'conexaopython'
engine = db.create_engine(f'mysql+{db_server}://{user}:{password}@{ip}:{db_port}/{db_name}?charset=utf8')
conn = engine.connect()

In [3]:
#localização do chromewebdriver
chrome_path = r'C:\Users\T-GAMER\Desktop\Ironhack DA\Projeto Final\chromedriver'
ser = Service(chrome_path)

In [4]:
#chromewebdriver deve estar aberto para o codigo funcionar
driver = webdriver.Chrome(service=ser)

# Base coleta de dados primeira pagina

In [11]:
url = 'https://loft.com.br/imovel/apartamento-rua-francisco-rossano-vila-prudente-sao-paulo-1-quarto-40m2/ycf79w'

In [12]:
driver.get(url)

In [13]:
driver.current_url

'https://loft.com.br/imovel/apartamento-rua-francisco-rossano-vila-prudente-sao-paulo-1-quarto-40m2/ycf79w'

In [ ]:
#metodo do webdriver 'find_element para procurar um elemento dentro da minha URL pelo xpath' 
#peguei o caminho full_xpath do valor no grande box a direita
driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div/ul/li[2]/p').text

In [ ]:
#peguei o caminho full_xpath do ID do Imóvel nas letras cinzas embaixo das fotos
#será necessário limpar o DataFrame para criar uma chave unica deste ID
driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[9]/span').text

In [ ]:
#peguei o caminho full_xpath da rua nas letras cinzas embaixo das fotos 
driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[7]/a').text

In [ ]:
#peguei o caminho full_xpath do tipo do imovel nas descrições em preto embaixo das fotos 
#será necessário limpar o tipo com regex, seria uma boa ideia entender os tipos de imovel dentro do site e depois filtrar eles
driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[3]/div[2]/div[1]/p').text

In [ ]:
#peguei o caminho full_xpath do titulo nas descrições em preto embaixo das foto para extrair a quantidade de quartos, suites e vagas de gaaragem  
#será necessário limpar o tipo com regex
driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[2]/h1').text

In [ ]:
#peguei o caminho full_xpath das descrições para extrair a quantidade de banheiros.  
#será necessário limpar o tipo com regex
driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[3]/div[2]/div[5]/p').text

# Func de Coleta uma pagina unica

In [23]:

def coleta_info():
    
    """Essa funcao coleta os dados de um imovel na pagina de anuncio do site Loft por meio do Xpath e salva estes dados dentro de um dicionario
    Atualmente para a funcao ser executada corretamente é necessario carregar a pagina previamente no navegador de maneira manual
    Eventualmente o site atualiza de um dia para o outro e os caminhos de Xpath devem ser atualizados"""
    
    
    # Imputar verificacao para caso ele não encontre uma das informações preencher como nulo o campo no caso # 
    
    preco_imov = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div[1]/ul/li[1]/p').text
    id_imov = driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[9]/span').text
    cidade = driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[3]/a').text
    endereco = driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[7]/a').text
    titulo_imov = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[2]/h1').text
    vaga = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[3]/div[3]/div[4]/p').text
    banheiro = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[3]/div[3]/div[5]/p').text
    etc_part = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[3]/div[4]/div[2]/div/span').text 
    condominio = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div[1]/ul/div[2]/li[2]/div[2]/p').text
    iptu = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div[1]/ul/div[2]/li[3]/div[2]/p').text
    
    global dict_teste
    dict_teste = {'Preco': preco_imov, 'Id': id_imov, 'Cidade': cidade, 'Endereco': endereco, 'Descricoes Imovel': titulo_imov, 'Vagas': vaga, 'Banheiros': banheiro, 
                'Outras Infos': etc_part, 'Custo Condominio': condominio, 'Iptu': iptu}


In [25]:
dict_teste

{'Preco': 'R$ 299.000',
 'Id': 'Imóvel YCF79W',
 'Cidade': 'Apartamento em São Paulo',
 'Endereco': 'Rua Francisco Rossano',
 'Descricoes Imovel': 'Apartamento à venda em Vila Prudente com 40 m², 1 quarto, 1 suíte, 1 vaga',
 'Vagas': '1 vaga',
 'Banheiros': '1 Banheiro',
 'Outras Infos': 'Possui lavabo',
 'Custo Condominio': 'R$ 360/mês',
 'Iptu': 'Isento'}

In [ ]:
df = pd.DataFrame(dict_teste,index = [0])

In [ ]:
df

# Func de automatização do codigo para rodar em varias paginas.

In [28]:
# coleta de lista de links da home do site

# home_url_loop = f'https://loft.com.br/venda/imoveis/sp/sao-paulo?pagina={i}'  #inicia na 2 e vai até a 132

home_url = f'https://loft.com.br/venda/imoveis/sp/sao-paulo?pagina={i}'
home_response = requests.get(home_url)
html = home_response.content
soup = BeautifulSoup(html)

# List comprehantion que junta o url da home filtrando dentro da tag <a> todas as tags <href> que iniciam em '/imovel'

lista_links = ['https://loft.com.br' + tag ['href'] for tag in soup.find_all('a') if tag['href'].startswith('/imovel')]


In [123]:
# For que cria uma lista de todos os links de imoveis listados para sao paulo

lista_links = []

for i in range(1,132):
    home_url = f'https://loft.com.br/venda/imoveis/sp/sao-paulo?pagina={i}'
    home_response = requests.get(home_url)
    html = home_response.content
    soup = BeautifulSoup(html)
    lista_links += ['https://loft.com.br' + tag ['href'] for tag in soup.find_all('a') if tag['href'].startswith('/imovel')]



In [127]:
lista_links

['https://loft.com.br/imovel/apartamento-padre-machado-bosque-da-saude-sao-paulo-2-quartos-124m2/rohcie',
 'https://loft.com.br/imovel/apartamento-piracuama-perdizes-sao-paulo-2-quartos-63m2/k5k3ug',
 'https://loft.com.br/imovel/apartamento-cristiano-viana-pinheiros-sao-paulo-2-quartos-69m2/3uziia',
 'https://loft.com.br/imovel/apartamento-doutor-alfredo-de-castro-perdizes-sao-paulo-2-quartos-57m2/qvuddw',
 'https://loft.com.br/imovel/apartamento-dos-americanos-bom-retiro-sao-paulo-2-quartos-57m2/leguzv',
 'https://loft.com.br/imovel/apartamento-eugenio-de-freitas-vila-guilherme-sao-paulo-2-quartos-48m2/8bnt0e',
 'https://loft.com.br/imovel/apartamento-tucuna-perdizes-sao-paulo-2-quartos-54m2/o0i86n',
 'https://loft.com.br/imovel/apartamento-francisco-leitao-pinheiros-sao-paulo-1-quarto-44m2/1pl7h10',
 'https://loft.com.br/imovel/apartamento-da-independencia-cambuci-sao-paulo-3-quartos-98m2/17llheb',
 'https://loft.com.br/imovel/apartamento-jaguaribe-higienopolis-sao-paulo-1-quarto-35m